
# EDA de *Televisión Penetración Totales* (con ID)

**Objetivo:** Notebook para VS Code (Python) con celdas *Markdown* entre pasos que:
1) **cargue** el dataset desde la ruta indicada,
2) **perfilar** y **limpie** los datos,
3) **visualice** tendencias clave,
4) **guarde** el CSV **limpio** agregando la leyenda `" limpio"` al nombre.



## Paso 0 — Configuración
- Imports, estilo de gráficos y definición de **ruta**.
- `data_path` usa tu **ruta Windows**. Si no existe, se usa el **archivo local** adjunto.


In [ ]:

# === Configuración ===
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8')

data_path = r"C:\\Users\\u607935\\OneDrive - Telecom Argentina SA\\Escritorio\\Curso de Data Analitycs\\Proyencto Integrador Final Grupo 7\\TV\\Televisión Penetración Totales_con_ID.csv"
local_path = r"Televisión Penetración Totales_con_ID.csv"

def resolve_path(primary, fallback):
    if os.path.exists(primary):
        return primary
    elif os.path.exists(fallback):
        return fallback
    else:
        raise FileNotFoundError(f"No se encontró el archivo en '{primary}' ni en '{fallback}'.")

resolved_path = resolve_path(data_path, local_path)
print('Usando archivo:', resolved_path)



## Paso 1 — Carga del dataset
- Lectura del CSV con `pandas`.
- Se inicia con `dtype=string` para controlar formatos y luego se convierte.


In [ ]:

dtypes_str = {
    'Registro_ID': 'string',
    'Año': 'string',
    'Trimestre': 'string',
    'TV por Suscripción (VFR) C/100 hab': 'string',
    'TV Satelital C/100 hab': 'string',
    'TV por Suscripción (VFR) C/100 hog': 'string',
    'TV Satelital C/100 hog': 'string',
    'TiempoTrim_ID': 'string',
}

df_raw = pd.read_csv(resolved_path, dtype=dtypes_str, encoding='utf-8-sig')
print('Shape:', df_raw.shape)
df_raw.head()



## Paso 2 — Perfilado inicial
- Columnas, tipos, nulos y duplicados.
- Distribución de años y trimestres.


In [ ]:

print('Columnas:', list(df_raw.columns))
print('Tipos:
', df_raw.dtypes)
print('Nulos por columna:
', df_raw.isna().sum())
print('Duplicados (filas completas):', df_raw.duplicated().sum())

print('Años únicos:', sorted(df_raw['Año'].dropna().unique()))
print('Trimestres únicos:', sorted(df_raw['Trimestre'].dropna().unique()))



## Paso 3 — Limpieza y normalización
Acciones:
1. **Renombrar** columnas a nombres cortos.
2. **Convertir** numéricos (coma/punto) a `float`.
3. **Tipificar** `anio`, `trimestre`, claves `registro_id` y `tiempo_trim_id`.
4. **Crear** la columna de **fecha** (inicio del trimestre) y ordenar.


In [ ]:

# 1) Renombrado
rename_map = {
    'Registro_ID': 'registro_id',
    'Año': 'anio',
    'Trimestre': 'trimestre',
    'TV por Suscripción (VFR) C/100 hab': 'tv_vfr_hab',
    'TV Satelital C/100 hab': 'tv_sat_hab',
    'TV por Suscripción (VFR) C/100 hog': 'tv_vfr_hog',
    'TV Satelital C/100 hog': 'tv_sat_hog',
    'TiempoTrim_ID': 'tiempo_trim_id',
}

df = df_raw.rename(columns=rename_map).copy()

# 2) Conversión numérica robusta: reemplazar coma -> punto y eliminar espacios
num_cols = ['tv_vfr_hab','tv_sat_hab','tv_vfr_hog','tv_sat_hog']
for c in num_cols:
    s = df[c].astype(str).str.replace(' ','', regex=False).str.replace(' ', '', regex=False)
    s = s.str.replace(',', '.', regex=False)  # por si viniera con coma decimal
    df[c] = pd.to_numeric(s, errors='coerce')

# 3) Tipificar claves y tiempo
for col in ['anio','trimestre']:
    df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')

# registro_id y tiempo_trim_id como enteros opcionales
for col in ['registro_id','tiempo_trim_id']:
    df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')

# 4) Fecha de inicio del trimestre
q_start_month = {1:1, 2:4, 3:7, 4:10}
df['fecha'] = pd.to_datetime(dict(year=df['anio'].astype('int'),
                                  month=df['trimestre'].map(q_start_month).astype('int'),
                                  day=1), errors='coerce')

# Orden
df = df.sort_values(['anio','trimestre']).reset_index(drop=True)

print(df.dtypes)
df.head()



## Paso 4 — Estadística descriptiva
- Resumen de métricas clave post-limpieza.


In [ ]:

df[['tv_vfr_hab','tv_sat_hab','tv_vfr_hog','tv_sat_hog']].describe().T



## Paso 5 — Visualizaciones
- Evolución **/100 hab** (VFR vs Satelital).
- Evolución **/100 hog** (VFR vs Satelital).


In [ ]:

import matplotlib.pyplot as plt
fig, axes = plt.subplots(2, 1, figsize=(10, 8), sharex=True)
axes[0].plot(df['fecha'], df['tv_vfr_hab'], label='VFR /100 hab', color='tab:blue')
axes[0].plot(df['fecha'], df['tv_sat_hab'], label='Satelital /100 hab', color='tab:orange')
axes[0].set_title('Penetración por 100 habitantes')
axes[0].set_ylabel('Penetración')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(df['fecha'], df['tv_vfr_hog'], label='VFR /100 hog', color='tab:green')
axes[1].plot(df['fecha'], df['tv_sat_hog'], label='Satelital /100 hog', color='tab:red')
axes[1].set_title('Penetración por 100 hogares')
axes[1].set_ylabel('Penetración')
axes[1].set_xlabel('Fecha (inicio de trimestre)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()



## Paso 6 — Tendencias (QoQ & YoY)
- **QoQ**: diferencia respecto del trimestre inmediato anterior.
- **YoY**: diferencia contra el mismo trimestre del año previo (shift 4).


In [ ]:

for c in ['tv_vfr_hab','tv_sat_hab','tv_vfr_hog','tv_sat_hog']:
    df[c+'_qoq'] = df[c].diff()
    df[c+'_yoy'] = df[c] - df[c].shift(4)

cols_show = [
    'anio','trimestre',
    'tv_vfr_hab','tv_vfr_hab_qoq','tv_vfr_hab_yoy',
    'tv_sat_hab','tv_sat_hab_qoq','tv_sat_hab_yoy',
    'tv_vfr_hog','tv_vfr_hog_qoq','tv_vfr_hog_yoy',
    'tv_sat_hog','tv_sat_hog_qoq','tv_sat_hog_yoy'
]

df[cols_show].tail(10)



## Paso 7 — Exportación del **CSV limpio**
- Se guarda en el directorio del archivo original (si existe), o en el directorio actual.
- El nombre se construye insertando la leyenda `" limpio"` antes de la extensión.


In [ ]:

import os

def make_clean_name(path):
    base_dir = os.path.dirname(path)
    base_name = os.path.basename(path)
    name, ext = os.path.splitext(base_name)
    clean_name = f"{name} limpio{ext if ext else '.csv'}"
    return os.path.join(base_dir if base_dir else os.getcwd(), clean_name)

clean_path = make_clean_name(resolved_path)
df.to_csv(clean_path, index=False, encoding='utf-8-sig')
print('CSV limpio guardado en:', clean_path)
